In [0]:
%run ./01-config

In [0]:
class HistoryLoader():
    def __init__(self,env):
        config = Config()
        self.silver_db = config.silver_db_name
        self.base_data_dir = config.base_dir_data + "/raw"
        self.base_data_dir_test = config.base_dir_data + "/test"
        self.checkpoint_dir = config.base_dir_checkpoint
        self.catalog = env


    def load_data_lookup(self):
        spark.sql(f"""INSERT OVERWRITE {self.catalog}.{self.silver_db}.date_lookup
                  SELECT date,dayofmonth,dayofweek,dayofyear,month,week,year,week_part FROM json.`{self.base_data_dir_test}/6-date-lookup.json`
                  """)

    def history_loader(self):
        self.load_data_lookup()
    	

    def assert_count(self,table_name,expected_count):
        actual_count = spark.read.table(f"{self.catalog}.{self.silver_db}.{table_name}").count()
        print(f"actual_count : {actual_count}")
        assert actual_count == expected_count, f"Expected {expected_count} rows in {table_name}, found {actual_count}"
        print(f"Expected {expected_count} rows in {table_name}, found {actual_count}")

    def validate(self):
        self.assert_count("date_lookup",365)
